**In this task** **<font color='red'>YOU</font>** will make machine translation system without using parallel corpora, alignment, attention, 100500 depth super-cool recurrent neural network and all that kind superstuff.

But even without parallel corpora this system can be good enough (hopefully), in particular for similar languages.

## Data

In this notebook we're going to use pretrained word vectors - FastText (original paper - https://arxiv.org/abs/1607.04606).

You can download them from the official [website](https://fasttext.cc/docs/en/crawl-vectors.html). We're going to need embeddings for English and French languages.

In [ ]:
!wget -nc https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
!gzip -d cc.en.300.vec.gz

!wget -nc https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fr.300.vec.gz
!gzip -d cc.fr.300.vec.gz

--2023-04-22 21:22:33--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.249.141.9, 13.249.141.40, 13.249.141.108, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.249.141.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1325960915 (1.2G) [binary/octet-stream]
Saving to: ‘cc.en.300.vec.gz’

cc.en.300.vec.gz    100%[===================>]   1.23G   183MB/s    in 7.0s    

2023-04-22 21:22:40 (182 MB/s) - ‘cc.en.300.vec.gz’ saved [1325960915/1325960915]

--2023-04-22 21:23:34--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fr.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 108.156.201.23, 108.156.201.129, 108.156.201.76, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|108.156.201.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1287757366 (1.2G) [binary/octet-stream]
Saving to

After downloading and extracting the vectors, we should be able to load them using the [gensim](https://radimrehurek.com/gensim/) library:

In [ ]:
from gensim.models import KeyedVectors
import numpy as np

In [ ]:
en_emb = KeyedVectors.load_word2vec_format("cc.en.300.vec")
fr_emb = KeyedVectors.load_word2vec_format("cc.fr.300.vec")

Once you've loaded the vectors, you can use the `KeyedVectors` interface to get word embeddings and/or query most similar words by embedding:

In [ ]:
august_embedding = en_emb["august"]
august_embedding.shape, august_embedding[:5]


((300,), array([-0.0522,  0.0364, -0.1252,  0.0053,  0.0382], dtype=float32))

In [ ]:
en_emb.most_similar([august_embedding], topn=10)

[('august', 0.9999999403953552),
 ('september', 0.8252838850021362),
 ('october', 0.8111193180084229),
 ('june', 0.8050147891044617),
 ('july', 0.797055184841156),
 ('november', 0.788363516330719),
 ('february', 0.7831973433494568),
 ('december', 0.7824540138244629),
 ('january', 0.7743154168128967),
 ('april', 0.7621643543243408)]

Everything above is true for the embeddings for French language.

In [ ]:
fr_emb.most_similar([fr_emb["aout"]])

[('aout', 1.0),
 ('Aout', 0.8249964118003845),
 ('juillet', 0.8109882473945618),
 ('fevrier', 0.8072442412376404),
 ('septembre', 0.7838520407676697),
 ('août', 0.779176652431488),
 ('juin', 0.7692081332206726),
 ('octobre', 0.7597455382347107),
 ('decembre', 0.7595790028572083),
 ('avril', 0.7390779256820679)]

However, french and english embeddings were trained independently of each other. This means, that there is no obvious connection between values in embeddings for similar words in French and English:

In [ ]:
fr_emb.most_similar([en_emb["august"]])

[('2003Pays', 0.23082853853702545),
 ('Montsoriu', 0.22505579888820648),
 ('2015Pays', 0.22218400239944458),
 ('2013Genre', 0.2095685601234436),
 ('AdiCloud', 0.2018650770187378),
 ('Bagua', 0.20061466097831726),
 ('2003Paysans', 0.2001495361328125),
 ('ValenceLa', 0.2001476287841797),
 ('Luddites', 0.19998176395893097),
 ('Guadalquivir', 0.19875513017177582)]

## Translation

We'll build a simple translator, which will try to predict the french embedding from the english one. For this we'll need a dataset of word pairs.

In [ ]:
def load_word_pairs(filename):
    en_fr_pairs = []
    en_vectors = []
    fr_vectors = []
    with open(filename, "r") as inpf:
        for line in inpf:
            en, fr = line.rstrip().split(" ")
            if en not in en_emb or fr not in fr_emb:
                continue
            en_fr_pairs.append((en, fr))
            en_vectors.append(en_emb[en])
            fr_vectors.append(fr_emb[fr])
    return en_fr_pairs, np.array(en_vectors), np.array(fr_vectors)

We will train our model to predict embedding for the french word from embedding of its english counterpart. For this reason we split our train and test data into english and french words and compute corresponding embeddings to obtain `X` (english embeddings) and `y` (french embeddings).

In [ ]:
!wget -O en-fr.train.txt https://raw.githubusercontent.com/girafe-ai/ml-course/23s_nes/homeworks/hw04_umt/en-fr.train.txt
!wget -O en-fr.test.txt https://raw.githubusercontent.com/girafe-ai/ml-course/23s_nes/homeworks/hw04_umt/en-fr.test.txt

In [ ]:
en_fr_train, X_train, Y_train = load_word_pairs("en-fr.train.txt")
en_fr_test, X_test, Y_test = load_word_pairs("en-fr.test.txt")

In [ ]:
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

In [ ]:
en_fr_train[33:44]

Let $x_i \in \mathrm{R}^d$ be the distributed representation of word $i$ in the source language, and $y_i \in \mathrm{R}^d$ is the vector representation of its translation. Our purpose is to learn such linear transform $W$ that minimizes euclidian distance between $Wx_i$ and $y_i$ for some subset of word embeddings. Thus we can formulate so-called [Procrustes problem](https://en.wikipedia.org/wiki/Orthogonal_Procrustes_problem):

$$W^*= \arg\min_W \sum_{i=1}^n\|Wx_i - y_i\|_2$$

or

$$W^*= \arg\min_W \|XW^T - Y\|_F$$

where $\|\cdot\|_F$ denotes Frobenius norm.

> **Note:** in second formula, $W$ and $x$ seem to have switched places. This happens because the $X$ matrix is composed of objects $x_i$ in *rows* not *columns*, i.e. it is kind of composed of $x_i^T$. This means that $X \in \mathbb{R}^{N \times D}$, where $N$ is the number of items and $D$ is the embedding dimensionality. The same is true for the $Y$.

$W^*= \arg\min_W \sum_{i=1}^n\|Wx_i - y_i\|_2$ looks like simple multiple linear regression without bias. The `sklearn` allows you to turn off the bias in `LinearRegression` via the `fit_intercept` argument (in fact they simply call bias the intercept). So let's code.

In [ ]:
from sklearn.linear_model import LinearRegression


mapping = LinearRegression(fit_intercept=True).fit(X_train, Y_train)

In [ ]:
mapping.intercept_.shape 

Let's take a look at neigbours of the vector of word _"august"_ (_"aout"_ in French) after linear transform.

In [ ]:
august_dot = mapping.coef_ @ en_emb["august"] + mapping.intercept_  #.reshape(1, -1)
fr_emb.most_similar(august_dot, topn=20)

In [ ]:
august = mapping.predict(en_emb["august"].reshape(1, -1))
fr_emb.most_similar(august, topn=20)

In [ ]:
from nltk.tokenize import WordPunctTokenizer
tk = WordPunctTokenizer()

In [ ]:
def translate(sentence):
    """
    :args:
        sentence - sentence in English (str)
    :returns:
        translation - sentence in French (str)

    * find english embedding for each word in sentence
    * transform english embedding vector
    * find nearest french word and replace
    """
    tokens = tk.tokenize(sentence)
    # your_code here
    

    return " ".join(translated)

In [ ]:
assert translate(".") == "."
assert translate("I walk around Paris") == "je marcher autour Paris"

In [ ]:
translate("I walk around Paris hhahahahah")

In [ ]:
tk.tokenize("I walk ., around Paris hhahahahah")

In [ ]:
en_emb[tk.tokenize("I walk ., around Paris hhahahahah")]